In [8]:
from gurobipy import Model, GRB, quicksum

# Initialize the model
model = Model("VRPSPD")

# Define sets
nodes = [0, 1, 2, 3, 4]  # 0 is the depot
customer_nodes = [1, 2, 3, 4]  # Excluding depot
vehicles = [0, 1]  # Two vehicles
K = len(vehicles)  # Total number of vehicles

# Example data
distance = {(i, j): 10 for i in nodes for j in nodes if i != j}  # Dummy distances
capacity = 100  # Vehicle capacity Q
pickup = {1: 10, 2: 15, 3: 20, 4: 25}
delivery = {1: 5, 2: 10, 3: 15, 4: 20}
M1 = 1000  # Big-M constant for delivery
M2 = 1000  # Big-M constant for load

# Decision Variables
x = model.addVars(nodes, nodes, vtype=GRB.BINARY, name="x")
D = model.addVars(nodes, vtype=GRB.CONTINUOUS, name="D")  # Delivery load
L = model.addVars(nodes, vtype=GRB.CONTINUOUS, name="L")  # Total load

# Prevent self-loops
for i in nodes:
    model.addConstr(x[i, i] == 0, name=f"no_self_loop_{i}")

# **Constraint 1: Each customer node is left by exactly one vehicle**
for j in customer_nodes:
    model.addConstr(quicksum(x[i, j] for i in nodes if i != j) == 1, name=f"leave_once_{j}")

# **Constraint 2: Each customer node is approached by exactly one vehicle**
for j in customer_nodes:
    model.addConstr(quicksum(x[j, i] for i in nodes if i != j) == 1, name=f"approach_once_{j}")

# **Constraint 3: At most `K` vehicles leave the depot**
model.addConstr(quicksum(x[0, i] for i in customer_nodes) <= K, name="limit_vehicles_leaving_depot")

# **Constraint 4: Delivery load consistency**
for i in customer_nodes:
    for j in customer_nodes:
        model.addConstr(D[i] >= D[j] + delivery[i] - M1 * (1 - x[i, j]), name=f"delivery_consistency_{i}_{j}")

# **Constraint 5: Load consistency at customer nodes**
for j in customer_nodes:
    model.addConstr(L[j] >= D[j] - delivery[j] + pickup[j], name=f"load_consistency_{j}")

# **Constraint 6: Load consistency between nodes**
for i in customer_nodes:
    for j in customer_nodes:
        model.addConstr(L[j] >= L[i] - delivery[j] + pickup[j] - M2 * (1 - x[i, j]), name=f"load_transfer_{i}_{j}")

# **Constraint 7: Delivery demand feasibility**
for i in customer_nodes:  # CHANGED: Iterate over customer_nodes instead of nodes
    model.addConstr(delivery[i] <= D[i], name=f"delivery_feasibility_lower_{i}")
    model.addConstr(D[i] <= capacity, name=f"delivery_feasibility_upper_{i}")

# **Constraint 8: Pickup demand feasibility**
for i in customer_nodes:  # CHANGED: Iterate over customer_nodes instead of nodes
    model.addConstr(pickup[i] <= L[i], name=f"pickup_feasibility_lower_{i}")
    model.addConstr(L[i] <= capacity, name=f"pickup_feasibility_upper_{i}")

# **Constraint 9: Binary constraint for vehicle routing**
for i in nodes:
    for j in nodes:
        model.addConstr(x[i, j] >= 0, name=f"binary_lb_{i}_{j}")
        model.addConstr(x[i, j] <= 1, name=f"binary_ub_{i}_{j}")

# **Constraint 10: Non-negativity of load variables**
for i in nodes:
    model.addConstr(D[i] >= 0, name=f"non_negative_D_{i}")
    model.addConstr(L[i] >= 0, name=f"non_negative_L_{i}")

# Objective: Minimize total distance traveled
model.setObjective(quicksum(distance[i, j] * x[i, j] for i in nodes for j in nodes if i != j), GRB.MINIMIZE)

# Solve Model
model.optimize()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 10.0 (19045.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1255U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Academic license 2632106 - for non-commercial use only - registered to a.___@student.tudelft.nl
Optimize a model with 126 rows, 35 columns and 205 nonzeros
Model fingerprint: 0x1eb61a73
Variable types: 10 continuous, 25 integer (25 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+01, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Found heuristic solution: objective 60.0000000
Presolve removed 89 rows and 11 columns
Presolve time: 0.00s
Presolved: 37 rows, 24 columns, 160 nonzeros
Variable types: 8 continuous, 16 integer (16 binary)

Root relaxation: objective 4.000000e+01, 19 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      

In [9]:
if model.status == GRB.OPTIMAL:
    print("\nOptimal Solution Found:")
    for var in model.getVars():
        print(f"{var.varName}: {var.x}")
    print(f"\nObjective Value: {model.objVal}")



Optimal Solution Found:
x[0,0]: 0.0
x[0,1]: 0.0
x[0,2]: 0.0
x[0,3]: 0.0
x[0,4]: 1.0
x[1,0]: 0.0
x[1,1]: 0.0
x[1,2]: 1.0
x[1,3]: 0.0
x[1,4]: 0.0
x[2,0]: 0.0
x[2,1]: 0.0
x[2,2]: 0.0
x[2,3]: 1.0
x[2,4]: 0.0
x[3,0]: 1.0
x[3,1]: 0.0
x[3,2]: 0.0
x[3,3]: 0.0
x[3,4]: 0.0
x[4,0]: 0.0
x[4,1]: 1.0
x[4,2]: 0.0
x[4,3]: 0.0
x[4,4]: 0.0
D[0]: 0.0
D[1]: 30.0
D[2]: 25.0
D[3]: 15.0
D[4]: 50.0
L[0]: 0.0
L[1]: 60.0
L[2]: 65.0
L[3]: 70.0
L[4]: 55.0

Objective Value: 50.0


In [4]:
if model.status == GRB.OPTIMAL:
    print("\n--- Verification Results ---")
    for i in nodes:
        for j in nodes:
            for v in vehicles:
                if x[i, j, v].x > 0.5:
                    print(f"Vehicle {v} travels from {i} to {j}")
    print(f"Total Cost: {model.objVal}")



--- Verification Results ---


KeyError: (0, 0, 0)

In [5]:
for cap in [80, 100, 120]:  # Testing different vehicle capacities
    print(f"\nTesting with Capacity = {cap}")
    model.reset()  # Reset model before each test
    for v in vehicles:
        model.addConstr(sum(y[i] for i in nodes) <= cap)
    model.optimize()



Testing with Capacity = 80
Discarded solution information


NameError: name 'y' is not defined